In [1]:
import ipywidgets as widgets
from stravalib.client import Client
from datetime import datetime
from stravalib.exc import AccessUnauthorized
import matplotlib.pyplot as plt
import folium
import branca
import numpy as np
from IPython.display import display, HTML
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import logging
# Set logging level to suppress informational messages from stravalib
logging.getLogger('stravalib').setLevel(logging.ERROR)
import sys
import io
import re
import requests
from IPython.display import display, HTML
import dateutil.parser
import calendar

## Route to Biodiversity

**Route to Biodiversity** aims to spotlight the rich biodiversity along athletic routes, emphasizing the close ties between human activities and surrounding ecosystems, and advocating for a harmonious coexistence with nature.

- **Route:** Users input their [Strava](http://www.strava.com) activity, which sets the geographic parameters for our analysis.
- **Biodiversity:** Leveraging the [iNaturalist](http://www.iNaturalist.org) database, we gather data on species observed in proximity to the specified Strava activity.

An integral aspect of **Route to Biodiversity** is the acknowledgment and recognition of indigenous territories, retrieved from [Native Land](https://native-land.ca/).

In [2]:
# StravaOAuth Class
class StravaOAuth:
    def __init__(self, client_id, client_secret, redirect_uri):
        self.client_id = client_id
        self.client_secret = client_secret
        self.redirect_uri = redirect_uri
        self.access_token = None
        self.refresh_token = None
        self.expires_at = None
    
    def get_authorization_url(self, scopes):
        oauth_url = f"https://www.strava.com/oauth/authorize?client_id={self.client_id}&response_type=code&redirect_uri={self.redirect_uri}&scope={scopes}"
        return oauth_url

    def exchange_token(self, code):
        payload = {
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'code': code,
            'grant_type': 'authorization_code'
        }
        response = requests.post("https://www.strava.com/oauth/token", data=payload)
        if response.status_code == 200:
            resp_json = response.json()
            self.access_token = resp_json['access_token']
            self.refresh_token = resp_json['refresh_token']
            self.expires_at = resp_json['expires_at']

# Global instance
strava = StravaOAuth("99300", "3807d10d20fc27ccaf92f90bd7f51ec4c7d077cf", "http://localhost/callback")

# Step 1: Direct the athlete to Strava's authorization page
SCOPES = "activity:read"
# SCOPES = "activity:read,activity:write"
oauth_url = strava.get_authorization_url(SCOPES)

# Create the HTML link
link_html = f'<a href="{oauth_url}" target="_blank">Click on the link to authorize on Strava</a>'
display(HTML(link_html))

# Step 2: Provide a widget for the athlete to paste the full callback URL from Strava
code_input = widgets.Text(value='', placeholder='Enter the full callback URL from Strava', description='Callback URL:')
display(code_input)

def on_code_input_change(change):
    url = change['new']
    match = re.search(r'code=([\w\d]+)', url)
    if match:
        code = match.group(1)
        code_input.value = code
        strava.exchange_token(code)

code_input.observe(on_code_input_change, names='value')


Text(value='', description='Callback URL:', placeholder='Enter the full callback URL from Strava')

In [2]:
# Functions
def route_to_bbox(route_coordinates):
    lats, lngs = zip(*route_coordinates)
    min_lat, max_lat = min(lats), max(lats)
    min_lng, max_lng = min(lngs), max(lngs)
    return (min_lat, min_lng, max_lat, max_lng)

def get_indigenous_territories_from_bbox(bbox):
    average_lat = (bbox[0] + bbox[2]) / 2
    average_lon = (bbox[1] + bbox[3]) / 2
    api_url = f"https://native-land.ca/api/index.php?maps=territories&position={average_lat},{average_lon}"
    response = requests.get(api_url)
    territories = response.json()
    territory_info = [{"name": territory["properties"]["Name"], "url": territory["properties"]["description"]} for territory in territories]
    return territory_info

# Step 3: Provide a widget for the athlete to paste the activity URL
activity_url_input = widgets.Text(value='', placeholder='Paste your Strava activity URL here', description='Activity URL:')
display(activity_url_input)

# Step 4: Fetch the details of the pasted activity
output = widgets.Output()
display(output)
activity = []
month = []
route_coords = []
bbox_route = []

def fetch_activity_data(button):
    global activity, month
    global route_coords  
    global bbox_route  
    with output:
        output.clear_output()
        activity_id = activity_url_input.value.split('/')[-1]
        activity_endpoint = f"activities/{activity_id}"
        streams_endpoint = f"activities/{activity_id}/streams?keys=latlng&key_by_type=true"
        
        # API Request: Get Activity
        detailed_activity = requests.get(
            f"https://www.strava.com/api/v3/{activity_endpoint}", 
            headers={'Authorization': f'Bearer {strava.access_token}'}
        ).json()
        activity = detailed_activity
        activity_date_str = activity['start_date_local']
        activity_date = dateutil.parser.parse(activity['start_date_local'])
        month_act = activity_date.month
        month = month_act
        print(f"Activity Name: {detailed_activity.get('name', 'Unknown')}")
        
        # API Request: Get Streams
        streams = requests.get(
            f"https://www.strava.com/api/v3/{streams_endpoint}", 
            headers={'Authorization': f'Bearer {strava.access_token}'}
        ).json()
        
        if 'latlng' in streams:
            route_coordinates = streams['latlng']['data']
            bbox = route_to_bbox(route_coordinates)
            bbox_route = bbox
            print(f"Geographic coordinates: {bbox}")
            route_coords = route_coordinates
            try:
                indigenous_territories_info = get_indigenous_territories_from_bbox(bbox)
                territories_str = ", ".join([f"<a href='{info['url']}'>{info['name']}</a>" for info in indigenous_territories_info])
                table_title = f"The land of: {territories_str}"
                display(HTML(table_title))
            except Exception as e:
                table_title = "The land of a colonized group (search for it!)."
                print(table_title)
                print(f"Exception encountered: {e}")
        else:
            print("Could not fetch route data for the activity.")
        return activity, month

# Display fetch button
fetch_button = widgets.Button(description="Fetch Activity Data")
fetch_button.on_click(fetch_activity_data)
display(fetch_button)
# Extract the activity date and parse the month


NameError: name 'widgets' is not defined

In [4]:
# Mapping verbose country names to their shorter version
COUNTRY_NAME_MAPPING = {
    "United States of America (the)": "United States"
    # Add other mappings here as needed
}

def identify_country_from_bbox(bbox):
    # Use the center of the bbox to get country information
    lat_center = (bbox[0] + bbox[2]) / 2
    lng_center = (bbox[1] + bbox[3]) / 2

    BASE_URL = "https://api.bigdatacloud.net/data/reverse-geocode-client"
    params = {
        "latitude": lat_center,
        "longitude": lng_center,
        "localityLanguage": "en"
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        country = response.json().get("countryName")
        return COUNTRY_NAME_MAPPING.get(country, country)
    return None


results_all = []
def fetch_species_from_inaturalist(bbox, month, taxa="any", species_count=10, order="desc"):
    global results_all
    BASE_URL = "https://api.inaturalist.org/v1/observations/species_counts"

    params = {
        "d1": "2000-01-01",
        "d2": "today",
        "geo": "true",
        "place_id": "any",
        "verifiable": "true",
        "iconic_taxa": taxa,
        "swlat": bbox[0],
        "swlng": bbox[1],
        "nelat": bbox[2],
        "nelng": bbox[3],
        "order": order,
        "order_by": "observations",
        "per_page": species_count,
        "month": month  # Add month to the parameters
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f"Failed to fetch data from iNaturalist. Status code: {response.status_code}")
        return []
    
    results = response.json()["results"]
    for species in results:
        if 'default_photo' in species['taxon'] and 'square_url' in species['taxon']['default_photo']:
            species["thumbnail_url"] = species['taxon']['default_photo']['square_url']
    results_all = results
    return results

def fetch_top_species_from_inaturalist(bbox):
    country = identify_country_from_bbox(bbox_route)
    if not country:
        print("Failed to identify the country from the provided bounding box.")
        return

    taxa_list = ['Mammalia', 'Reptilia', 'Aves', 'Plantae', 'Amphibia', 'Fungi', 'Insecta', 'Arachnida']

    # Fetch data for each taxa
    taxa_data = {}
    for taxa in taxa_list:
        taxa_data[taxa] = fetch_species_from_inaturalist(bbox_route, month, taxa=taxa, species_count=10, order="desc")

    # Display results as a minimalistic table with thumbnails and common names
    month_name = calendar.month_name[month] if month is not None else ""
    month_str = f" in {month_name}" if month_name else ""
    table_html = '<table style="width:100%;">'
    table_html += f'<tr><th colspan="{len(taxa_list)}" style="text-align:center; font-size:1.5em;">Top species observed along your route{month_str}</th></tr>'  # Title row
    table_html += '<tr>'
    for taxa in taxa_list:
        table_html += f'<th style="text-align:center;">{taxa}</th>'
    table_html += '</tr>'

    for i in range(10):
        columns = []
        for taxa in taxa_list:
            entry = ""
            data = taxa_data[taxa]
            if i < len(data):
                taxon = data[i]["taxon"]
                thumbnail_url = data[i]["thumbnail_url"]
                species_name = taxon["name"]
                common_name = taxon.get("preferred_common_name", "")
                species_link = f'https://www.inaturalist.org/taxa/{taxon["id"]}'
                entry += f'<div style="text-align:center;"><a href="{species_link}" target="_blank">{common_name}</a></div>'
                entry += f'<a href="{species_link}" target="_blank"><img src="{thumbnail_url}" style="display:block; margin:auto; width:auto; max-height:100px;" alt="{species_name}"></a>'
                entry += f'<a href="{species_link}" target="_blank" style="display:block; text-align:center;">{species_name}</a>'
            columns.append(entry)
        table_html += '<tr>'
        for column in columns:
            table_html += f'<td style="text-align:center;">{column}</td>'
        table_html += '</tr>'

    table_html += '</table>'
    return table_html


def plot_route_and_bbox_on_map(route_coordinates, bbox):
    # Create a map centered around the middle of the route
    center_lat = (bbox[0] + bbox[2]) / 2
    center_lng = (bbox[1] + bbox[3]) / 2
    m = folium.Map(location=[center_lat, center_lng], zoom_start=15, tiles='Stamen Terrain')

    # Plot the route on the map
    folium.PolyLine(route_coordinates, color="blue", weight=2.5, opacity=1).add_to(m)

    # Plot the bounding box
    bbox_coordinates = [
        (bbox[0], bbox[1]),
        (bbox[0], bbox[3]),
        (bbox[2], bbox[3]),
        (bbox[2], bbox[1]),
        (bbox[0], bbox[1])
    ]
    folium.PolyLine(bbox_coordinates, color="red", weight=2.5, opacity=1, dash_array=[8, 4]).add_to(m)
    m.fit_bounds([(bbox[0], bbox[1]), (bbox[2], bbox[3])])

    # Create a fixed legend on the map
    legend_html = """
         <div style="position: fixed; 
                     bottom: 50px; right: 50px; width: 200px; height: 90px; 
                     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
                     &nbsp;<b>Legend</b><br>
                     &nbsp;<span style="color:blue;">●</span>&nbsp;Activity Route<br>
                     &nbsp;<span style="color:red;">▬</span>&nbsp;Observation Area<br>
          </div>
         """
    m.get_root().html.add_child(folium.Element(legend_html))

    display(m)
    
output_widget = widgets.Output()



def fetch_and_plot(button):
    with output_widget:
        # Clear previous outputs
        output_widget.clear_output()
        
        # Fetch and display species
        table_html = fetch_top_species_from_inaturalist(bbox_route)
        display(HTML(table_html))
        
        # Plot the route and bounding box on map
        plot_route_and_bbox_on_map(route_coords, bbox_route)

# Create the button
get_species_button = widgets.Button(description="Get species")

# Bind the function to the button
get_species_button.on_click(fetch_and_plot)

# Display the button and output widget
display(get_species_button)
display(output_widget)

Button(description='Get species', style=ButtonStyle())

Output()

In [5]:
import ipywidgets as widgets
from IPython.display import display, HTML
import calendar
import requests
import folium

# Mapping verbose country names to their shorter version
COUNTRY_NAME_MAPPING = {
    "United States of America (the)": "United States"
    # Add other mappings here as needed
}

def identify_country_from_bbox(bbox):
    lat_center = (bbox[0] + bbox[2]) / 2
    lng_center = (bbox[1] + bbox[3]) / 2

    BASE_URL = "https://api.bigdatacloud.net/data/reverse-geocode-client"
    params = {
        "latitude": lat_center,
        "longitude": lng_center,
        "localityLanguage": "en"
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        country = response.json().get("countryName")
        return COUNTRY_NAME_MAPPING.get(country, country)
    return None

results_all = []
def fetch_species_from_inaturalist(bbox, month=None, taxa="any", species_count=10, order="desc"):
    BASE_URL = "https://api.inaturalist.org/v1/observations/species_counts"
    params = {
        "d1": "2000-01-01",
        "d2": "today",
        "geo": "true",
        "place_id": "any",
        "verifiable": "true",
        "iconic_taxa": taxa,
        "swlat": bbox[0],
        "swlng": bbox[1],
        "nelat": bbox[2],
        "nelng": bbox[3],
        "order": order,
        "order_by": "observations",
        "per_page": species_count
    }

    # Only add month to params if it is not None
    if month is not None:
        params["month"] = month
    
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f"Failed to fetch data from iNaturalist. Status code: {response.status_code}")
        return []
    
    results = response.json()["results"]
    
    for species in results:
        if 'default_photo' in species['taxon'] and 'square_url' in species['taxon']['default_photo']:
            species["thumbnail_url"] = species['taxon']['default_photo']['square_url']
    return results

def fetch_top_species_from_inaturalist(bbox, month, normalize=False):
    country = identify_country_from_bbox(bbox_route)
    if not country:
        print("Failed to identify the country from the provided bounding box.")
        return

    taxa_list = ['Mammalia', 'Reptilia', 'Aves', 'Plantae', 'Amphibia', 'Fungi', 'Insecta', 'Arachnida']
    taxa_data = {}
    for taxa in taxa_list:
        taxa_data[taxa] = fetch_species_from_inaturalist(bbox_route, month, taxa=taxa, species_count=10, order="desc", normalize=normalize)

    month_name = calendar.month_name[month] if month is not None else ""
    month_str = f" in {month_name}" if month_name else ""
    table_html = '<table style="width:100%;">'
    table_html += f'<tr><th colspan="{len(taxa_list)}" style="text-align:center; font-size:1.5em;">Top species observed along your route{month_str}</th></tr>'
    table_html += '<tr>'
    for taxa in taxa_list:
        table_html += f'<th style="text-align:center;">{taxa}</th>'
    table_html += '</tr>'

    for i in range(10):
        columns = []
        for taxa in taxa_list:
            entry = ""
            data = taxa_data[taxa]
            if i < len(data):
                taxon = data[i]["taxon"]
                thumbnail_url = data[i]["thumbnail_url"]
                species_name = taxon["name"]
                common_name = taxon.get("preferred_common_name", "")
                species_link = f'https://www.inaturalist.org/taxa/{taxon["id"]}'
                entry += f'<div style="text-align:center;"><a href="{species_link}" target="_blank">{common_name}</a></div>'
                entry += f'<a href="{species_link}" target="_blank"><img src="{thumbnail_url}" style="display:block; margin:auto; width:auto; max-height:100px;" alt="{species_name}"></a>'
                entry += f'<a href="{species_link}" target="_blank" style="display:block; text-align:center;">{species_name}</a>'
            columns.append(entry)
        table_html += '<tr>'
        for column in columns:
            table_html += f'<td style="text-align:center;">{column}</td>'
        table_html += '</tr>'

    table_html += '</table>'
    return table_html

def plot_route_and_bbox_on_map(route_coordinates, bbox):
    center_lat = (bbox[0] + bbox[2]) / 2
    center_lng = (bbox[1] + bbox[3]) / 2
    m = folium.Map(location=[center_lat, center_lng], zoom_start=15, tiles='Stamen Terrain')
    folium.PolyLine(route_coordinates, color="blue", weight=2.5, opacity=1).add_to(m)

    bbox_coordinates = [
        (bbox[0], bbox[1]),
        (bbox[0], bbox[3]),
        (bbox[2], bbox[3]),
        (bbox[2], bbox[1]),
        (bbox[0], bbox[1])
    ]
    folium.PolyLine(bbox_coordinates, color="red", weight=2.5, opacity=1, dash_array=[8, 4]).add_to(m)
    m.fit_bounds([(bbox[0], bbox[1]), (bbox[2], bbox[3])])

    legend_html = """
         <div style="position: fixed; 
                     bottom: 50px; right: 50px; width: 200px; height: 90px; 
                     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
                     &nbsp;<b>Legend</b><br>
                     &nbsp;<span style="color:blue;">●</span>&nbsp;Activity Route<br>
                     &nbsp;<span style="color:red;">▬</span>&nbsp;Observation Area<br>
          </div>
         """
    m.get_root().html.add_child(folium.Element(legend_html))

    display(m)

# Assume month_act is defined somewhere in your notebook, as per your file
# month_act = ...

# Ensure month is defined and within the valid range
  # or whatever default value you deem appropriate

month_dropdown = widgets.Dropdown(
    options=[("Year long", None)] + [(calendar.month_name[i], i) for i in range(1, 13)], 
    value=None,
    description='Month:',
)

normalize_checkbox = widgets.Checkbox(
    value=False,
    description='Normalize by Total Year Count',
    disabled=False
)

output_widget = widgets.Output()

def fetch_and_plot(button):
    with output_widget:
        # Clear previous outputs
        output_widget.clear_output()
        
        # Fetch and display species
        table_html = fetch_top_species_from_inaturalist(bbox_route, month=month_dropdown.value)
        display(HTML(table_html))
        
        # Plot the route and bounding box on map
        plot_route_and_bbox_on_map(route_coords, bbox_route)

get_species_button = widgets.Button(description="Get species")
get_species_button.on_click(fetch_and_plot)

display(month_dropdown)
display(get_species_button)
display(output_widget)

Dropdown(description='Month:', index=9, options=(('January', 1), ('February', 2), ('March', 3), ('April', 4), …

Checkbox(value=False, description='Normalize by Total Year Count')

Button(description='Get species', style=ButtonStyle())

Output()

In [ ]:
https://www.strava.com/activities/10003219945

## User Instructions

1. **Strava Authorization**:
- Click on the provided link to authorize with Strava.
- You will be redirected to Strava's login page. Log in using your Strava credentials.
- After logging in, Strava will ask for permission to share your activity data. Grant the necessary permissions.
- You will be redirected to a callback URL. ***This website is broken (This site can’t be reached, localhost refused to connect). That's normal.*** You just need to copy the entire URL from your browser's address bar.

2. **Enter the Callback URL**:
- Return to the ***Route to Biodiversity*** tool.
- In the "Callback" field, paste the copied URL.

3. **Enter Your Strava Activity URL**:
- Navigate to your desired activity on Strava's website.
- Copy the URL of that activity from your browser's address bar.
- Return to the ***Route to Biodiversity*** tool and paste the activity URL in the provided field.

4. **Fetch Species Data**:
- Click the "Get species" button.
- The tool will analyze the area around your Strava activity and display the top 10 species most frequently observed in the vicinity.

5. **Explore the Results**:
- View the highlighted biodiversity along your athletic route.
- For more detailed information about specific species, click on their respective links or entries.


**Disclaimer:** The data presented in **Route to Biodiversity** is not exhaustive. It encompasses observations from the year 2000 up to the present day. Typically, the observations reflect species that individuals find interesting or those that are more easily observed during daylight hours. It's important to note that the number of observations listed for a particular species does not equate to the number of existing individuals of that species. Instead, it represents the number of observations of that species that have been uploaded to [**iNaturalist**](https://www.inaturalist.org/) from that specific area. While every effort is made to provide accurate and comprehensive information, there may be species that are underrepresented or not represented at all due to these and other factors. We encourage users to approach the data with an understanding of these limitations and to use it as a starting point for further exploration and appreciation of the rich biodiversity around them.